In [3]:
from typing import Dict, List


class Graph2Sat:
    def __init__(self, fname: str=None):
        self.G = {}
        self.Grev = {}
        self.NodeRange = 0
        if fname:
            self.read_file(fname)
    
    def read_file(self, fname: str='SCC.txt'):
        with open(fname) as f:
            self.NodeRange = int(f.readline())
            while line := f.readline():
                if line == '\n':
                    continue
                tail, head = list(map(int, line.split('\n')[0].split(' ')[:2]))
                for tail, head in [(-tail, head), (-head, tail)]:
                    if tail not in self.G:
                        self.G[tail] = []
                    self.G[tail].append(head)
                    if head not in self.Grev:
                        self.Grev[head] = []
                    self.Grev[head].append(tail)
        
    def dfs_loop_rev(self) -> Dict[int, int]:
        visited = set()
        stack = []
        order = 0
        f = {}
        for node in range(-self.NodeRange, self.NodeRange + 1):
            if node in visited:
                continue
            stack.append(node)
            
            while stack:
                has_children = False
                next_node = stack[-1]
                visited.add(next_node)    # !!!everything we add to stack we add to visited
                for child in self.Grev.get(next_node, []):
                    if child not in visited:
                        stack.append(child)
                        visited.add(child)    # !!!everything we add to stack we add to visited
                        # we will delete items from the stack only after we have seen all the children
                        has_children = True   
                if not has_children:
                    poped = stack.pop()
                    order += 1
                    f[order] = poped
        return f
    
    def dfs_loop(self, f: Dict[int, int]) -> Dict[int, List[int]]:
        visited = set()
        stack = []
        scc_dict = {}
        for i in range(self.NodeRange, 0, -1):
            node = f[i]
            if node in visited:
                continue
            stack.append(node) # !!!everything we add to stack we add to visited
            visited.add(node)
            scc_dict[node] = set()
            while stack:
                next_node = stack.pop()
                visited.add(next_node)
                for child in self.G.get(next_node, []):
                    if child not in visited:
                        stack.append(child)
                        scc_dict[node].add(child)
                        visited.add(child) # !!!everything we add to stack we add to visited
        return scc_dict
    
    def find_SCC(self) -> Dict[int, List[int]]:
        f = self.dfs_loop_rev()
        scc_dict = self.dfs_loop(f)
        return scc_dict
    
    def find_SCC_lens(self) -> List[int]:
        lens = list(sorted([len(item) + 1 for item in self.find_SCC().values()], reverse=True))
        return lens

In [9]:
for i in range(1, 7):
    fname = f'./tests/2sat{i}.txt'
    twosat = Graph2Sat(fname)
    print(fname)
    scc_dict = twosat.find_SCC()
    for key, scc in scc_dict.items():
        for num in scc:
            if -num in scc:
                print(num)
                

./tests/2sat1.txt
./tests/2sat2.txt
-162741
-76873
-9187
9187
76873
162741
./tests/2sat3.txt
./tests/2sat4.txt
./tests/2sat5.txt
80428
144085
-220453
-80428
220453
-144085
618692
-618692
./tests/2sat6.txt
273101
-410420
410420
-273101
